# sentenceとwordのデータのまとめ

In [1]:
rosbag_dir = "/home/miyazawa/SIGVerse/sigros_ws/src/rosbag_database/src/bag/"

In [2]:
import sys
import glob
import os
import re
import numpy as np
import pandas as pd

sys.path.append("/opt/ros/indigo/lib/python2.7/dist-packages")
sys.path.append("/opt/tmc/ros/indigo/lib/python2.7/dist-packages")
sys.path.append("/home/miyazawa/SIGVerse/sigros_ws/devel/lib/python2.7/dist-packages")
sys.path.append("/home/miyazawa/catkin_ws/devel/lib/python2.7/dist-packages")
                
import rosbag
import rospy
from std_msgs.msg import  Header
from audio_module_msg.msg import AudioSentence

No handlers could be found for logger "rosout"


In [3]:
rosbag_path_list = glob.glob(os.path.join(rosbag_dir,"*/*"))
rosbag_path_list.sort()

* rosbag sentence to DataFrame  
データの書式をcaptionと合わせるように，secの精度でstampを取得

In [4]:
rosbag_df = pd.DataFrame(columns=["stamp","sentence"])
for rosbag_path in rosbag_path_list:
    bag = rosbag.Bag(rosbag_path)
    values = []
    for topic, msg, t in bag.read_messages('/AudioSentence'):
        values.append([int(msg.header.stamp.to_sec()), msg.sentences[0]])
    tmp_df = pd.DataFrame(data=values, columns=["stamp","sentence"])
    rosbag_df = rosbag_df.append(tmp_df, ignore_index=True)

* captionのデータを収集

In [5]:
caption_dir = glob.glob("/home/miyazawa/SIGVerse/data/0411/caption/exp*")
convert = lambda text: int(text) if text.isdigit() else text
alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
caption_dir.sort(key = alphanum_key)

In [6]:
values = []
action_nums = []
for exp_id, caption_path in enumerate(caption_dir):
    timestamp_path = glob.glob(caption_path + '/time*')
    timestamp_path.sort()
    for action_id, path in enumerate(timestamp_path):
        try:
            for stamp in np.loadtxt(path):
                values.append([int(stamp), exp_id, action_id])
        except:
            values.append([int(stamp), exp_id, action_id])
    action_nums.append(action_id)
caption_df = pd.DataFrame(data=values,columns=["stamp", "exp_num", "action_num"])

* spco の word　を収集

In [7]:
spco_word_path = glob.glob("/home/miyazawa/SIGVerse/sigros_ws/src/spco_formation/training_data/trial/word/*.txt")
spco_word_stamp_path = glob.glob("/home/miyazawa/SIGVerse/sigros_ws/src/spco_formation/training_data/trial/word/stamp/*")
convert = lambda text: int(text) if text.isdigit() else text
alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
spco_word_path.sort(key = alphanum_key)
spco_word_stamp_path.sort(key = alphanum_key)

In [8]:
values = []
for path,stamp_path in zip(spco_word_path,spco_word_stamp_path):
    index = int(path.split("/")[-1].strip(".txt"))
    word = np.loadtxt(path,dtype=str)
    stamp = int(np.loadtxt(stamp_path)) / pow(10,9)
    values.append([index,word,stamp])
df1 = pd.DataFrame(values,columns=["data_id","word","stamp"])

In [9]:
iteration_num = 10
values = []
count = 0
trial_num = 10
output_file = "/home/miyazawa/SIGVerse/sigros_ws/src/spco_formation/result/output_{0}.txt".format(trial_num)
for sen in np.loadtxt(output_file,dtype=str,delimiter="\n"):
    if "Iteration: {0}".format(iteration_num) in sen:
        class_id = int(sen.split()[-1])
        values.append([count,class_id])
        count += 1
df2 = pd.DataFrame(values,columns=["data_id","class_id"])

In [10]:
tmp = pd.merge(df1,df2,on="data_id",how="outer")
tmp = tmp.dropna()

* データのマージ

In [17]:
sentence_df = pd.merge(rosbag_df, caption_df, on="stamp", how="outer")
sentence_df = pd.merge(sentence_df, tmp , on="stamp", how="outer")
sentence_df = sentence_df.sort_values(by="stamp")
sentence_df = sentence_df.dropna()

In [18]:
caption_df = caption_df.sort_values(by=["stamp","exp_num","action_num"])
caption_df[caption_df.stamp.duplicated()]

,stamp,exp_num,action_num
2,1522664645,0,1
33,1522665677,3,2
34,1522665685,3,2
97,1522667936,10,2
105,1522668151,11,1
106,1522668159,11,1


In [19]:
caption_df.sort_values(by="stamp")

,stamp,exp_num,action_num
0,1522664634,0,0
1,1522664645,0,0
2,1522664645,0,1
3,1522664661,0,1
4,1522664680,0,1
5,1522664739,0,2
6,1522664763,0,2
7,1522664772,0,2
8,1522664907,1,0
9,1522664916,1,0


In [21]:
place_hist = []
for exp_num, action_num_max in enumerate(action_nums):
    for action_num in range(action_num_max + 1):
        tmp_place_hist = np.zeros(100)
        tmp_df = sentence_df[(sentence_df.exp_num==exp_num) & (sentence_df.action_num==action_num)]
        for i in tmp_df.groupby(by="class_id").count().word.iteritems():
            class_id = int(i[0])
            num = i[1]
            tmp_place_hist[class_id] = num
        place_hist.append(tmp_place_hist)
place_hist = np.array(place_hist)

In [22]:
np.savetxt("/home/miyazawa/SIGVerse/sigros_ws/src/mmlda_bhmm/LearnData/place.Place",place_hist,fmt="%d")

In [23]:
sentence_df[sentence_df.word=="キッチン"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
25,1.522665e+09,キッチンの机に設置した,1.0,2.0,64.0,キッチン,98.0
56,1.522666e+09,キッチンの棚に設置,3.0,3.0,139.0,キッチン,98.0
89,1.522667e+09,キッチンの棚に設置,6.0,2.0,266.0,キッチン,23.0
90,1.522667e+09,キッチンの棚に設置した,6.0,2.0,267.0,キッチン,98.0
115,1.522667e+09,キッチンの棚に置いた,8.0,2.0,337.0,キッチン,98.0
142,1.522668e+09,キッチンの棚に置いた,10.0,3.0,413.0,キッチン,98.0
171,1.522669e+09,キッチンの棚に置いた,12.0,1.0,501.0,キッチン,98.0


In [24]:
sentence_df[sentence_df.word=="風呂"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
10,1.522665e+09,お風呂の棚に置く,0.0,2.0,28.0,風呂,52.0
11,1.522665e+09,お風呂の棚に置いた,0.0,2.0,29.0,風呂,97.0
37,1.522665e+09,お風呂の棚に置いた,2.0,1.0,96.0,風呂,52.0
67,1.522666e+09,お風呂の棚に設置する,4.0,1.0,176.0,風呂,52.0
69,1.522666e+09,お風呂の棚に,4.0,1.0,177.0,風呂,52.0
70,1.522666e+09,お風呂の棚に設置した,4.0,1.0,178.0,風呂,97.0
99,1.522667e+09,お風呂の棚に設置した,7.0,1.0,297.0,風呂,52.0
100,1.522667e+09,お風呂の棚に置いた,7.0,1.0,299.0,風呂,98.0
123,1.522668e+09,お風呂の棚に置く,9.0,1.0,368.0,風呂,52.0
126,1.522668e+09,お風呂の棚に置く,9.0,1.0,369.0,風呂,52.0


In [25]:
sentence_df[sentence_df.word=="寝室"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
2,1.522665e+09,寝室で飲み物を握る,0.0,0.0,3.0,寝室,98.0
3,1.522665e+09,寝室で飲み物を握る,0.0,1.0,3.0,寝室,98.0
5,1.522665e+09,寝室で飲み物を掴んだ,0.0,1.0,4.0,寝室,98.0
18,1.522665e+09,寝室で飲み物を把持する,1.0,1.0,46.0,寝室,98.0
19,1.522665e+09,寝室で飲み物を掴んだ,1.0,1.0,47.0,寝室,98.0
29,1.522665e+09,寝室で ぬいぐるみを把持する,2.0,0.0,69.0,寝室,97.0
30,1.522665e+09,寝室で ぬいぐるみを掴んだ,2.0,0.0,70.0,寝室,98.0
31,1.522665e+09,ここは 寝室だよ,2.0,0.0,71.0,寝室,98.0
50,1.522666e+09,寝室で ぬいぐるみをはじした,3.0,2.0,119.0,寝室,98.0
60,1.522666e+09,寝室で果物を握る,4.0,0.0,144.0,寝室,98.0


In [26]:
sentence_df[sentence_df.word=="ドア前"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
15,1.522665e+09,ドア前でドアを押し開く,1.0,0.0,38.0,ドア前,98.0
41,1.522666e+09,ドア前でドアを押し開く,3.0,0.0,106.0,ドア前,22.0
42,1.522666e+09,ドア前でドアを開いた,3.0,0.0,107.0,ドア前,98.0
73,1.522667e+09,ドア前でドアを押す,5.0,0.0,188.0,ドア前,22.0
75,1.522667e+09,ドア前でドアを開ける,5.0,0.0,189.0,ドア前,98.0
82,1.522667e+09,ドア前でドアを押し開ける,6.0,0.0,214.0,ドア前,98.0
103,1.522667e+09,ドア前でドアを開く,8.0,0.0,308.0,ドア前,22.0
104,1.522667e+09,ドア前でドアを押す,8.0,0.0,309.0,ドア前,98.0
130,1.522668e+09,ドア前でドアを開く,10.0,0.0,379.0,ドア前,98.0
